# Linear regression

The Boston housing prices dataset

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import time

from sklearn.datasets import load_boston

data = load_boston()
X = data.data
y = data.target

In [ ]:
print(data.DESCR)

### Linear regression

$x^{(i)}\in\mathbb{R}^n$ is the $\text{i}^{\text{th}}$ training example.

$$
h(x) = w_0 + w_1\cdot x_1 + w_2\cdot x_2 + \ldots + w_n\cdot x_n
$$

Sometimes the input vector $x$ is extended as follows: $x = (1, x_1, x_2, \dotsc,x_n)$.


$$
h_{w} = w^T\cdot x
$$

Another notation that can be used:
$$
h(x) = b + w_1\cdot x_1 + w_2\cdot x_2 + \ldots + w_n\cdot x_n = w^T\cdot x + b.
$$

The cost function (objective function) that we would like to minimize:

$$
J(w) = \frac{1}{2m}\sum_{i=1}^{m}\left(h_w(x^{(i)}) - y^{(i)}\right)^2 = \frac{1}{2m}\sum_{i=1}^{m}\left(\hat y^{(i)} - y^{(i)}\right)^2 = \frac12 \text{MSE}
$$

Gradient descent method.

$$
w_j := w_j - \alpha\cdot \frac{\partial J(w)}{\partial w_j} \qquad (j=0, 1, 2, \dotsc, n),
$$
that is,

$$
w_j := w_j - \alpha \frac{1}{m} \sum_{i=1}^m (h_w(x^{(i)}) - y^{(i)}) x^{(i)}_j \qquad (j=0, 1, 2, \dotsc, n)
$$

In [ ]:
w1 = 1.45
w2 = -0.85
b = 3.21

np.random.seed(1)
X = 10 * np.random.rand(120, 2)
X = np.concatenate((np.ones((120, 1)), X), axis=1)

w = [b, w1, w2]
y = np.matmul(X, w) + 0.1 * np.random.randn(120)

X_train, X_test = X[:100, :], X[100:, :]
y_train, y_test = y[:100], y[100:]

$$
w = X^+y = (X^T\cdot X)^{-1}X^Ty,
$$

where $w = X^+y$ is the vector that solves $Xw = y$ in the least-square sense. 

In [ ]:
# Both of these give the same result

coeffs = np.linalg.solve(np.matmul(X_train.T, X_train), np.matmul(X_train.T, y_train))
print(coeffs)

coeffs = np.matmul(np.linalg.pinv(X_train), y_train)
print(coeffs)

In [ ]:
def predict(x, w):
    return np.dot(x, w)


def compute_cost(X, y, w):
    m = len(X)
    cost = 0
    for x_i, y_i in zip(X, y):
        y_i_hat = predict(x_i, w)
        cost += (y_i_hat - y_i) ** 2
    return cost / (2 * m)

In [ ]:
def calc_gradient(X, y, w):
    m = len(X)
    
    grad = np.empty_like(w)
    for j in range(len(w)):
        dw_j = 0
        for x_i, y_i in zip(X, y):
            y_i_hat = predict(x_i, w)
            dw_j += ((y_i_hat - y_i) * x_i[j])
        grad[j] = dw_j / m
    return grad


def gradient_descent(X, y, alpha, nr_iterations):
    _, nr_features = X.shape
    w = np.zeros(nr_features)
    costs = []
    
    for _ in range(nr_iterations):
        cost = compute_cost(X, y, w)
        costs.append(cost)
        dw = calc_gradient(X, y, w)
        w = w - alpha * dw
    return w, costs

In [ ]:
t = time.time()
weights, costs = gradient_descent(X, y, alpha=0.01, nr_iterations=10000)
print(f'Running time: {time.time() - t}')
print()
print(weights)

In [ ]:
figure = plt.figure(figsize=(16, 8))
plt.plot(costs)
plt.yscale('log')
plt.show()

In [ ]:
def predict(X, w):
    return np.matmul(X, w)


def compute_cost(X, y, w):
    m = len(X)
    y_hat = predict(X, w)
    return (1 / (2*m)) * np.sum(np.square(y_hat - y))


def calc_gradient(X, y, w):
    m = len(X)
    y_hat = predict(X, w)
    return (1 / m) * np.sum(np.multiply(X, np.expand_dims(y_hat - y, 1)), axis=0)


def gradient_descent(X, y, alpha, nr_iterations):
    m, nr_features = X.shape
    w = np.zeros(nr_features)
    costs = []
    
    for _ in range(nr_iterations):
        cost = compute_cost(X, y, w)
        costs.append(cost)
        dw = calc_gradient(X, y, w)
        w = w - alpha * dw
    return w, costs

In [ ]:
t = time.time()
weights, costs = gradient_descent(X, y, alpha=0.01, nr_iterations=10000)
print(f'Running time: {time.time() - t}')
print()
print(weights)

In [ ]:
figure = plt.figure(figsize=(16, 8))
plt.plot(costs)
plt.yscale('log')
plt.show()

### Regularization

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Generate data

np.random.seed(1)
data_size = 50

xs = np.sort(1.5 * np.random.random(data_size))
ys = 2 * np.power(xs, 3) + 1 + np.random.normal(scale=0.5, size=data_size)

In [ ]:
plt.figure(figsize=(16, 8))
plt.scatter(xs, ys)
plt.show()

In [ ]:
xs_train, xs_test, y_train, y_test = train_test_split(xs, ys, test_size=0.4)

In [ ]:
def add_powers(x, n):
    X = np.empty(shape=(len(x), n+1), dtype=np.float_)
    for j in range(n+1):
        X[:, j] = np.power(x, j)
    return X

In [ ]:
X_train = add_powers(xs_train, 12)
X_test = add_powers(xs_test, 12)

In [ ]:
coeffs = np.matmul(np.linalg.pinv(X_train), y_train)
print(coeffs)

y_hat = predict(X_train, coeffs)
cost = compute_cost(X_train, y_train, coeffs)
print(cost)

In [ ]:
plt.figure(figsize=(16, 8))
plt.scatter(xs_train, y_train)
plt.scatter(xs_train, y_hat)
plt.show()

In [ ]:
y_hat_test = predict(X_test, coeffs)

cost = compute_cost(X_test, y_test, coeffs)

print(cost)

In [ ]:
plt.figure(figsize=(16, 8))
plt.scatter(xs_test, y_test)
plt.scatter(xs_test, y_hat_test)
plt.show()

Idea: prevent large coefficients to eliminate higher order terms 

$$
J_{\lambda}(w) = \frac{1}{2m}\sum_{i=1}^{m}\left(h_w(x^{(i)}) - y^{(i)}\right)^2 + \lambda\cdot\sum_{j=1}^nw_j^2
$$

This modified cost function can also be minimized exactly:

$$
w = \left(X^TX + \lambda \cdot\text{diag}(0,1,1\dotsc,1)\right)^{-1}X^Ty
$$

In [ ]:
def lhs(X, lambda_):
    _, nr_cols = X.shape
    d = np.ones(nr_cols)
    d[0] = 0
    D = np.diag(d)
    return np.matmul(X_train.T, X_train) + lambda_ * D


lambda_ = 10
coeffs = np.linalg.solve(lhs(X_train, lambda_), np.matmul(X_train.T, y_train))
print(coeffs)


y_hat = predict(X_train, coeffs)
cost = compute_cost(X_train, y_train, coeffs)
print(cost)

In [ ]:
plt.figure(figsize=(16, 8))
plt.scatter(xs_train, y_train)
plt.scatter(xs_train, y_hat)
plt.show()

In [ ]:
y_hat_test = predict(X_test, coeffs)

cost = compute_cost(X_test, y_test, coeffs)
print(cost)

In [ ]:
plt.figure(figsize=(16, 10))
plt.scatter(xs_test, y_test)
plt.scatter(xs_test, y_hat_test)
plt.show()